# Отток клиентов

Из «Бета-Банка» стали уходить клиенты. Каждый месяц. Немного, но заметно. Банковские маркетологи посчитали: сохранять текущих клиентов дешевле, чем привлекать новых.

Нужно спрогнозировать, уйдёт клиент из банка в ближайшее время или нет. Вам предоставлены исторические данные о поведении клиентов и расторжении договоров с банком. 

Постройте модель с предельно большим значением *F1*-меры. Чтобы сдать проект успешно, нужно довести метрику до 0.59. Проверьте *F1*-меру на тестовой выборке самостоятельно.

Дополнительно измеряйте *AUC-ROC*, сравнивайте её значение с *F1*-мерой.

Источник данных: [https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling](https://www.kaggle.com/barelydedicated/bank-customer-churn-modeling)

# 1. Подготовка данных

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.utils import shuffle

In [2]:
df = pd.read_csv('/datasets/Churn.csv')

In [3]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2.0,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1.0,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8.0,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1.0,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2.0,125510.82,1,1,1,79084.10,0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
RowNumber          10000 non-null int64
CustomerId         10000 non-null int64
Surname            10000 non-null object
CreditScore        10000 non-null int64
Geography          10000 non-null object
Gender             10000 non-null object
Age                10000 non-null int64
Tenure             9091 non-null float64
Balance            10000 non-null float64
NumOfProducts      10000 non-null int64
HasCrCard          10000 non-null int64
IsActiveMember     10000 non-null int64
EstimatedSalary    10000 non-null float64
Exited             10000 non-null int64
dtypes: float64(3), int64(8), object(3)
memory usage: 1.1+ MB


##### Есть пропуски в столбце Tenure, посмотрим на них поближе

##### Заменим пропуски медианным значением. Тем более что есть клиенты, где в столбце Tenure указан 0, а значит предположить что вместо пропусков в данном случае подразумевается так же 0, будет неразумно

In [5]:
df.loc[df['Tenure'].isna(), 'Tenure'] = df.query('~Tenure.isna()')['Tenure'].median()

##### Выделим признаки и основной столбец. Избавимся от столбца RowNumber, он лишь помешает

In [6]:
features = df.drop(['RowNumber', 'Exited', 'CustomerId', 'Surname'], axis = 1)
target = df['Exited']

In [7]:
features = pd.get_dummies(features, drop_first = True)

##### Разделим фрейм на обучающую, валидационную и тестовую выборки

In [8]:
features_train, features_valid1, target_train, target_valid1 = train_test_split(features,
                                                    target,
                                                    train_size = 0.6,
                                                    random_state = 12345, 
                                                    stratify = target)

In [9]:
features_valid, features_test, target_valid, target_test = train_test_split(features_valid1,
                                                    target_valid1,
                                                    train_size = 0.5,
                                                    random_state = 12345,
                                                    stratify = target_valid1)

In [10]:
print('Размер тренировочной выборки:', features_train.shape, target_train.shape)
print('Размер валидационной выборки:', features_valid.shape, target_valid.shape)
print('Размер тестовой выборки:', features_test.shape, target_test.shape)

Размер тренировочной выборки: (6000, 11) (6000,)
Размер валидационной выборки: (2000, 11) (2000,)
Размер тестовой выборки: (2000, 11) (2000,)


##### Используем кодирование для количественных признаков

In [11]:
numeric = ['CreditScore', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'EstimatedSalary']

In [12]:
scaler = StandardScaler()
scaler.fit(features_train[numeric])

StandardScaler(copy=True, with_mean=True, with_std=True)

In [13]:
features_train[numeric] = scaler.transform(features_train[numeric])
features_valid[numeric] = scaler.transform(features_valid[numeric])
features_test[numeric] = scaler.transform(features_test[numeric])

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the ca

# 2. Исследование задачи

##### Рассмотрим accurscy_score трех моделей

In [14]:
model_DTC = DecisionTreeClassifier(random_state = 12345)
print(model_DTC.fit(features_train, target_train).score(features_valid, target_valid))

model_RFC = RandomForestClassifier(random_state = 12345, n_estimators = 100)
print(model_RFC.fit(features_train, target_train).score(features_valid, target_valid))

model_LR = LogisticRegression(random_state = 12345, solver = 'liblinear')
print(model_LR.fit(features_train, target_train).score(features_valid, target_valid))

0.793
0.8685
0.811


##### Самая высокая точность у модели случайного леса

In [15]:
target_train.value_counts(normalize = 1)

0    0.796333
1    0.203667
Name: Exited, dtype: float64

In [16]:
target_valid.value_counts(normalize = 1)

0    0.796
1    0.204
Name: Exited, dtype: float64

##### Присутствует большой дисбаланс классов

# 3. Борьба с дисбалансом

##### Посмотрим на матрицы ошибок для наших моделей

In [17]:
model_DTC = DecisionTreeClassifier(random_state=123)
model_DTC.fit(features_train, target_train)
DTC_predictions = model_DTC.predict(features_valid)
confusion_matrix(target_valid, DTC_predictions)

array([[1379,  213],
       [ 197,  211]])

##### Ложноположительных ответов, как и ложноотрицательных, столько же сколько и истинноположительныхю Посмотрим на другие метрики

In [18]:
def scores(target_valid, predictions):
    print("Полнота" , recall_score(target_valid, predictions))
    print("Точность", precision_score(target_valid, predictions))
    print("F1-мера", f1_score(target_valid, predictions))
    print("AUC-ROC", roc_auc_score(target_valid, predictions))

In [19]:
scores(target_valid, DTC_predictions)

Полнота 0.5171568627450981
Точность 0.49764150943396224
F1-мера 0.5072115384615384
AUC-ROC 0.6916814464479257


##### Низкое качество по большинству метрик

In [20]:
model_RFC = RandomForestClassifier(random_state = 12345, n_estimators = 100)
model_RFC.fit(features_train, target_train)
RFC_predictions = model_RFC.predict(features_valid)
confusion_matrix(target_valid, RFC_predictions)

array([[1529,   63],
       [ 200,  208]])

##### Ложных ответов меньше по сравнению с моделью дерева решений

In [21]:
scores(target_valid, RFC_predictions)

Полнота 0.5098039215686274
Точность 0.7675276752767528
F1-мера 0.6126656848306332
AUC-ROC 0.7351155286235097


##### Полнота немного хуже, но остальные параметры сильно выше

In [22]:
model_LR = LogisticRegression(solver = 'liblinear')
model_LR.fit(features_train, target_train)
LR_predictions = model_LR.predict(features_valid)
confusion_matrix(target_valid, LR_predictions)

array([[1538,   54],
       [ 324,   84]])

##### Очень много ложноотрицательных ответов

In [23]:
scores(target_valid, LR_predictions)

Полнота 0.20588235294117646
Точность 0.6086956521739131
F1-мера 0.3076923076923077
AUC-ROC 0.5859813774756133


##### Очень маленькая полнота и F1-мера

#### В изначальных данных присутствует сильный дисбаланс классов. Пока лучше всего себя показывает модель случайного леса. Чтобы найти подходящую модель для решения задачи, проведем ребаланс классов

##### Пришло время избавиться от дисбаланса классов. Как мы выяснили, почти в 80% случаев ответ отрицательный, значит увеличим количество положительных в 4 раза

In [24]:
def upsample(features, target, repeat):
    features_zeros = features[target == 0]
    features_ones = features[target == 1]
    target_zeros = target[target == 0]
    target_ones = target[target == 1]

    features_upsampled = pd.concat([features_zeros] + [features_ones] * repeat)
    target_upsampled = pd.concat([target_zeros] + [target_ones] * repeat)
    
    features_upsampled, target_upsampled = shuffle(
        features_upsampled, target_upsampled, random_state=12345)
    
    return features_upsampled, target_upsampled

In [25]:
features_train_upsample, target_train_upsample = upsample(features_train, target_train, 4)

In [26]:
def scores(features_train, target_train, features_valid, target_valid):
    model_DTC = DecisionTreeClassifier(random_state = 12345)
    print(model_DTC.fit(features_train, target_train).score(features_valid, target_valid))
    
    model_RFC = RandomForestClassifier(random_state = 12345, n_estimators = 100)
    print(model_RFC.fit(features_train, target_train).score(features_valid, target_valid))
    
    model_LR = LogisticRegression(random_state = 12345, solver = 'liblinear')
    print(model_LR.fit(features_train, target_train).score(features_valid, target_valid))

##### Посмотрим, как изменится точность на сбалансированной выборке. Что было, и что стало

In [27]:
scores(features_train, target_train, features_valid, target_valid)

0.793
0.8685
0.811


In [28]:
scores(features_train_upsample, target_train_upsample, features_valid, target_valid)

0.8065
0.862
0.7255


##### Точность упала, но это можно объяснить изменением баланса. Посмотрим на другие метрики

In [29]:
def score2(target_valid, predictions):
    print('Полнота:' , recall_score(target_valid, predictions))
    print('Точность:', precision_score(target_valid, predictions))
    print('F1-мера:', f1_score(target_valid, predictions))
    print('AUC-ROC:', roc_auc_score(target_valid, predictions))

##### Модель дерева решений

In [30]:
model_DTC_upsample = DecisionTreeClassifier(random_state = 12345)
model_DTC_upsample.fit(features_train_upsample, target_train_upsample)
predictions = model_DTC_upsample.predict(features_valid)
score2(target_valid, predictions)

Полнота: 0.5245098039215687
Точность: 0.5257985257985258
F1-мера: 0.5251533742331288
AUC-ROC: 0.701639324071337


##### Модель случайного леса

In [31]:
model_RFC_upsample = RandomForestClassifier(random_state = 12345, n_estimators = 100)
model_RFC_upsample.fit(features_train_upsample, target_train_upsample)
predictions = model_RFC_upsample.predict(features_valid)
score2(target_valid, predictions)

Полнота: 0.5906862745098039
Точность: 0.6885714285714286
F1-мера: 0.6358839050131926
AUC-ROC: 0.7611094689131934


##### Логистическая регрессия

In [32]:
model_LR_upsample = LogisticRegression(random_state = 12345, solver = 'liblinear')
model_LR_upsample.fit(features_train_upsample, target_train_upsample)
predictions = model_LR_upsample.predict(features_valid)
score2(target_valid, predictions)

Полнота: 0.7426470588235294
Точность: 0.40562248995983935
F1-мера: 0.5246753246753246
AUC-ROC: 0.7318762932308602


#### После ребаланса лучшие показатели у случайного леса. Найдем идеальные параметры для модели

In [33]:
max = 0
for i in range(1, 100):
    for j in range(5, 15):
        model = RandomForestClassifier(random_state = 12345, n_estimators = i, max_depth = j)
        model.fit(features_train_upsample, target_train_upsample)
        predictions = model.predict(features_valid)
        if f1_score(target_valid, predictions) > max:
            max = f1_score(target_valid, predictions)
            n = i
            m = j
print('Максимальный f1-score:', max, 'число деревьев:', n, 'максимальная глубина', m)

Максимальный f1-score: 0.6530612244897959 число деревьев: 37 максимальная глубина 10


##### С помощью двух циклов нашли оптимальные параметры для модели случайного леса - 37 деревьев, максимальная глубина - 10

##### Попробуем еще один метод борьбы с дисбалансом - уменьшение выборки

In [34]:
def downsample(features, target, fraction):
    features_zeros = features[target == 0]
    target_zeros = target[target == 0]
    features_ones = features[target == 1]
    target_ones = target[target == 1]
    
    features_downsample = pd.concat([features_zeros.sample(frac = fraction, random_state = 12345)] + [features_ones])
    target_downsample = pd.concat([target_zeros.sample(frac = fraction, random_state = 12345)] + [target_ones])
    
    features_downsample, target_downsample = shuffle(features_downsample, target_downsample, random_state = 12345)
    return features_downsample, target_downsample

In [35]:
features_train_downsample, target_train_downsample = downsample(features_train, target_train, 0.25)

In [36]:
model_DTC_downsample = DecisionTreeClassifier(random_state = 12345)
model_DTC_downsample.fit(features_train_downsample, target_train_downsample)
predictions = model_DTC_downsample.predict(features_valid)
score2(target_valid, predictions)

Полнота: 0.7181372549019608
Точность: 0.3737244897959184
F1-мера: 0.49161073825503354
AUC-ROC: 0.7048600847374126


In [37]:
model_RFC_downsample = RandomForestClassifier(n_estimators = 100, random_state = 12345)
model_RFC_downsample.fit(features_train_downsample, target_train_downsample)
predictions = model_RFC_downsample.predict(features_valid)
score2(target_valid, predictions)

Полнота: 0.7745098039215687
Точность: 0.4876543209876543
F1-мера: 0.5984848484848485
AUC-ROC: 0.7829835451768647


In [38]:
model_LR_downsample = LogisticRegression(random_state = 12345, solver = 'liblinear')
model_LR_downsample.fit(features_train_downsample, target_train_downsample)
predictions = model_LR_downsample.predict(features_valid)
score2(target_valid, predictions)

Полнота: 0.75
Точность: 0.40691489361702127
F1-мера: 0.5275862068965517
AUC-ROC: 0.7349246231155779


##### При уменьшении выборки случайный лес остается фаворитом, так что вернемся к ранее подобранным параметрам

# 4. Тестирование модели

In [39]:
model_RFC_upsample = RandomForestClassifier(max_depth = 10, n_estimators = 37, random_state = 12345)
model_RFC_upsample.fit(features_train_upsample, target_train_upsample)
predictions = model_RFC_upsample.predict(features_test)
score2(target_test, predictions)

Полнота: 0.6805896805896806
Точность: 0.5420743639921722
F1-мера: 0.6034858387799564
AUC-ROC: 0.7668485126112244


#### Итог:
##### В изначальных данных наблюдается сильный дисбаланс классов, примерно 4 к 1. После проведения ребаланса, лучше всего себя показала модель случайного леса с параметрами n_estimators = 37 и max_depth = 10.
##### У AUC-ROC довольно высокий показатель, как и у полноты, а значит модель охватывает бОльшую часть клиентов. А вот показатель точности невысок - примерно 0.55, значит модель верно предсказывает чуть больше половины уходов клиентов.
##### Модель лучше всего подойдет для того чтобы вычислить клиентов, которые могут уйти из банка в скором времени